In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation,Dropout,Flatten,Conv2D,MaxPool2D,Dense
from keras.models import load_model
from keras.preprocessing import image

c4 = cv2.imread('CATS_DOGS/train/CAT/4.jpg')
c4 = cv2.cvtColor(c4,cv2.COLOR_BGR2RGB)

d = cv2.imread('CATS_DOGS/train/DOG/2.jpg')
d= cv2.cvtColor(d,cv2.COLOR_BGR2RGB)

img_gen = ImageDataGenerator(rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            rescale=1/255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip = True,
                            fill_mode='nearest')

img_gen.flow_from_directory('CATS_DOGS/train')

input_shape=(150,150,3)
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),input_shape=(150,150,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.summary()

batch_size =16

train_img = img_gen.flow_from_directory('CATS_DOGS/train',
                                       target_size=input_shape[:2],
                                       batch_size=batch_size,
                                       class_mode='binary')

test_img = img_gen.flow_from_directory('CATS_DOGS/test',
                                       target_size=input_shape[:2],
                                       batch_size=batch_size,
                                       class_mode='binary')

result = model.fit_generator(train_img,epochs=100,steps_per_epoch=150,
                            validation_data=test_img,validation_steps=12)

result.history['ass']

new_model = load_model('cat_dog_100epochs.h5')

dog_file = 'CATS_DOGS/test/DOGS/10005.jpg'

dog_img = image.load_img(dog_file,target_size=(150,150))
dog_img = image.img_to_array(dog_img)

dof_img = np.expand_dims(dog_img,axis=0)
dog_img = dog_img/255
model.predict_classes(dog_img)
model.predict(dog_img)
